In [1]:
import pandas as pd

#RAW data as EXCEL
df = pd.read_excel("C:/Traffic Stops/traffic_stops/traffic_stops.xlsx")
print(df)

       stop_date stop_time country_name driver_gender  driver_age_raw  \
0     2020-01-01  00:00:00       Canada             M              59   
1     2020-01-01  00:01:00        India             M              35   
2     2020-01-01  00:02:00          USA             M              26   
3     2020-01-01  00:03:00       Canada             M              26   
4     2020-01-01  00:04:00       Canada             M              62   
...          ...       ...          ...           ...             ...   
65533 2020-02-15  12:13:00        India             F              54   
65534 2020-02-15  12:14:00       Canada             F              18   
65535 2020-02-15  12:15:00          USA             M              27   
65536 2020-02-15  12:16:00       Canada             F              49   
65537 2020-02-15  12:17:00          USA             F              72   

       driver_age driver_race     violation_raw violation  search_conducted  \
0              19       Asian     Drunk Driv

In [3]:
df = pd.read_excel("C:/Traffic Stops/traffic_stops/traffic_stops.xlsx")

# Drop unnecessary columns and fill missing values
df = df.drop(['violation_raw','driver_age_raw'], axis=1)
df = df.fillna("Unknown")

# Ensure stop_time and stop_date
df['stop_date'] = pd.to_datetime(df['stop_date'], errors='coerce').dt.strftime('%Y-%m-%d')
df['stop_time'] = pd.to_datetime(df['stop_time'], format='%H:%M:%S', errors='coerce').dt.time

# Optional: Save cleaned Excel
df.to_excel("cleaned_police_log.xlsx", index=False)
print(df)

        stop_date stop_time country_name driver_gender  driver_age  \
0      2020-01-01  00:00:00       Canada             M          19   
1      2020-01-01  00:01:00        India             M          58   
2      2020-01-01  00:02:00          USA             M          76   
3      2020-01-01  00:03:00       Canada             M          76   
4      2020-01-01  00:04:00       Canada             M          75   
...           ...       ...          ...           ...         ...   
65533  2020-02-15  12:13:00        India             F          48   
65534  2020-02-15  12:14:00       Canada             F          35   
65535  2020-02-15  12:15:00          USA             M          41   
65536  2020-02-15  12:16:00       Canada             F          63   
65537  2020-02-15  12:17:00          USA             F          32   

      driver_race violation  search_conducted     search_type stop_outcome  \
0           Asian  Speeding              True  Vehicle Search       Ticket   
1  

In [4]:
import pandas as pd 
#After remove nan value excel with name
# df.to_excel("cleaned_police_log.xlsx", index=False)

#cleaned file Path
df=pd.read_excel("C:/Traffic Stops/traffic_stops/cleaned_police_log.xlsx")
df

#printing the excel
df

,stop_date,stop_time,country_name,driver_gender,driver_age,driver_race,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,00:00:00,Canada,M,19,Asian,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,00:01:00,India,M,58,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,00:02:00,USA,M,76,Black,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,00:03:00,Canada,M,76,Black,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,00:04:00,Canada,M,75,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,48,Black,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568
65534,2020-02-15,12:14:00,Canada,F,35,Hispanic,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098
65535,2020-02-15,12:15:00,USA,M,41,Asian,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328
65536,2020-02-15,12:16:00,Canada,F,63,Black,Other,False,Unknown,Warning,True,0-15 Min,True,KA24UZ8488


In [64]:
# Connecting to MySQL
import streamlit as st
import pymysql

def create_connection():
    try:
        connection = pymysql.connect(
            host="localhost",
            user="root",
            password="deepi",
            database="traffic_stops"
        )
        connection.autocommit = True 
        return connection
    except Exception as e:
        st.error(f"Database connection failed: {e}")
        return None

connection = create_connection()
print("connection =", connection)    


connection = <pymysql.connections.Connection object at 0x000001A5D5C7D6D0>


In [ ]:
# --- Insert data ---
cursor = connection.cursor()
for _, row in df.iterrows():
    try:
        stop_date_str = str(pd.to_datetime(row['stop_date'], errors='coerce').date()) if pd.notnull(row['stop_date']) else None
        stop_time_str = str(pd.to_datetime(row['stop_time'], errors='coerce').time()) if pd.notnull(row['stop_time']) else None

        cursor.execute("""
            INSERT INTO police_post_log (
                stop_date, stop_time, country_name, driver_gender, driver_age,
                driver_race, violation, search_conducted, search_type,
                stop_outcome, is_arrested, stop_duration, drugs_related_stop,
                vehicle_number
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            stop_date_str, stop_time_str, row['country_name'], row['driver_gender'],
            row['driver_age'], row['driver_race'], row['violation'],
            row['search_conducted'], row['search_type'], row['stop_outcome'],
            row['is_arrested'], row['stop_duration'], row['drugs_related_stop'],
            row['vehicle_number']
        ))
    except Exception as e:
        print(" Error inserting police_log:", e)
        # print("Row data:", row.to_dict())

connection.commit()
cursor.close()
connection.close()
print("All rows inserted.")


All rows inserted.


In [33]:
# ------------------- fetch data -------------------
def fetch_data(query):
    connection = create_connection()
    if connection:
        try:
            with connection.cursor() as cursor:
                cursor.execute(query)
                result = cursor.fetchall()
                cols = [desc[0] for desc in cursor.description]
                return pd.DataFrame(result, columns=cols)
        finally:
            connection.close()
    return pd.DataFrame()